# Import modules

# Import modules

In [4]:
import tensorflow as tf
from facelib.dev.layer import DenseL2
from facelib.dev.loss import ArcFace
from facelib.dev.model import mobile_face_net


# Prepare model architecture

In [5]:
input_shape = (112,112,3)
num_classes = 500
num_features = 128
model = mobile_face_net(input_shape = input_shape)
x = model.output
x = tf.keras.layers.BatchNormalization(name='features')(x)
x = DenseL2(num_classes)(x)
model = tf.keras.models.Model(model.input, x)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


# Compile model

In [6]:
af = ArcFace(m=0.4, scale=30)
losses = [af.loss]
optimizer = tf.keras.optimizers.Adam()
metrics = [
    tf.keras.metrics.CategoricalAccuracy(name='acc'),
]
model.compile(optimizer=optimizer, loss=losses, metrics=metrics)


# Print model architecture

In [7]:
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 112, 112, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 56, 56, 64)        1792      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 56, 56, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 56, 56, 64)        256       
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 56, 56, 64)        640       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 56, 56, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 64)        256   

In [8]:
import numpy as np

In [9]:
np.random.randint(150)

105

In [10]:
type(np.random.randint(150))

int

In [11]:
np.eye(5, 5)

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [12]:
5 + (5,4)

TypeError: unsupported operand type(s) for +: 'int' and 'tuple'

In [13]:
5 + [5,4]

TypeError: unsupported operand type(s) for +: 'int' and 'list'

In [14]:
[1] + (5,4)

TypeError: can only concatenate list (not "tuple") to list

In [15]:
(1) + (5,4)

TypeError: unsupported operand type(s) for +: 'int' and 'tuple'

In [16]:
(1,) + (5,4)

(1, 5, 4)

In [17]:
batch_size = 64

In [18]:
dummy_label = np.random.randint(
    num_classes,
    (batch_size,)
) # shape = (64,500)


ValueError: low >= high

In [19]:
dummy_label = np.random.randint(
            0, num_classes,
            (batch_size,)
        ) # shape = (64,500)


In [20]:
dummy_label

array([446, 199, 240, 467, 451, 381, 272, 290, 213, 178, 138, 371, 198,
       433, 304, 252,  15, 127, 462, 377, 406, 134,   8, 345, 144, 137,
       418, 234, 328, 216, 363, 402, 202, 453, 435, 260, 476,  63, 107,
       125, 343, 122, 317, 334, 247, 337,  20, 429, 136, 129, 499, 174,
       353, 401, 114,  15,  44, 195, 321, 348, 272, 255,  74, 366])

In [21]:
dummy_label

array([446, 199, 240, 467, 451, 381, 272, 290, 213, 178, 138, 371, 198,
       433, 304, 252,  15, 127, 462, 377, 406, 134,   8, 345, 144, 137,
       418, 234, 328, 216, 363, 402, 202, 453, 435, 260, 476,  63, 107,
       125, 343, 122, 317, 334, 247, 337,  20, 429, 136, 129, 499, 174,
       353, 401, 114,  15,  44, 195, 321, 348, 272, 255,  74, 366])

In [22]:
np.eye(dummy_label)

TypeError: only integer scalar arrays can be converted to a scalar index

In [23]:
np.eye(dummy_label.reshape(-1))

TypeError: only integer scalar arrays can be converted to a scalar index

In [24]:
dummy

NameError: name 'dummy' is not defined

In [25]:
dumm

NameError: name 'dumm' is not defined

In [26]:
dummy_label

array([446, 199, 240, 467, 451, 381, 272, 290, 213, 178, 138, 371, 198,
       433, 304, 252,  15, 127, 462, 377, 406, 134,   8, 345, 144, 137,
       418, 234, 328, 216, 363, 402, 202, 453, 435, 260, 476,  63, 107,
       125, 343, 122, 317, 334, 247, 337,  20, 429, 136, 129, 499, 174,
       353, 401, 114,  15,  44, 195, 321, 348, 272, 255,  74, 366])

In [27]:
dummy_label.shape

(64,)

In [28]:
dummy_label.dtype

dtype('int64')

In [29]:
np.eye(500)[dummy_label]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [30]:
np.eye(50)[dummy_label]

IndexError: index 446 is out of bounds for axis 0 with size 50

In [31]:
np.eye(50)[[0,3,5]]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]])

In [32]:
np.eye(50)[[0,3,5]].shape

(3, 50)

In [33]:
np.eye(500)[dummy_label]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [34]:
np.eye(500)[dummy_label].shape

(64, 500)

# Dummy training data/label generator (batched)

In [1]:
epochs = 2
num_data = 50000
batch_size = 64
steps_per_epoch = num_data // batch_size

def data_gen():
    for i in range(steps_per_epoch):
        dummy_img = np.random.rand((batch_size,) + input_shape, dtype='float32')
        yield dummy_img # shape: (64,112,112,3)

def label_gen():
    for i in range(steps_per_epoch):
        dummy_label = np.random.randint(
            0, num_classes,
            (batch_size,)
        ) # shape: (64)
        one_hotted_label = np.eye(num_classes)[dummy_label]
        yield one_hotted_label # shape: (batch_size, num_classes)


In [2]:
for i in data_gen():
    print(i)
    break

NameError: name 'np' is not defined

In [3]:
import numpy as np

# Dummy training data/label generator (batched)

In [4]:
epochs = 2
num_data = 50000
batch_size = 64
steps_per_epoch = num_data // batch_size

def data_gen():
    for i in range(steps_per_epoch):
        dummy_img = np.random.rand((batch_size,) + input_shape, dtype='float32')
        yield dummy_img # shape: (64,112,112,3)

def label_gen():
    for i in range(steps_per_epoch):
        dummy_label = np.random.randint(
            0, num_classes,
            (batch_size,)
        ) # shape: (64)
        one_hotted_label = np.eye(num_classes)[dummy_label]
        yield one_hotted_label # shape: (batch_size, num_classes)


In [5]:
for i in data_gen():
    print(i)
    break

NameError: name 'input_shape' is not defined

In [6]:
input_shape = (112,112,3)

In [7]:
for i in data_gen():
    print(i)
    break

TypeError: rand() got an unexpected keyword argument 'dtype'

# Dummy training data/label generator (batched)

In [8]:
epochs = 2
num_data = 50000
batch_size = 64
steps_per_epoch = num_data // batch_size

def data_gen():
    for i in range(steps_per_epoch):
        dummy_img = np.random.rand((batch_size,) + input_shape).astype('float32')
        yield dummy_img # shape: (64,112,112,3)

def label_gen():
    for i in range(steps_per_epoch):
        dummy_label = np.random.randint(
            0, num_classes,
            (batch_size,)
        ) # shape: (64)
        one_hotted_label = np.eye(num_classes)[dummy_label]
        yield one_hotted_label # shape: (batch_size, num_classes)


In [9]:
for i in data_gen():
    print(i)
    break

TypeError: 'tuple' object cannot be interpreted as an integer

# Dummy training data/label generator (batched)

In [10]:
epochs = 2
num_data = 50000
batch_size = 64
steps_per_epoch = num_data // batch_size

def data_gen():
    for i in range(steps_per_epoch):
        dummy_img = np.random.rand(list((batch_size,) + input_shape)).astype('float32')
        yield dummy_img # shape: (64,112,112,3)

def label_gen():
    for i in range(steps_per_epoch):
        dummy_label = np.random.randint(
            0, num_classes,
            (batch_size,)
        ) # shape: (64)
        one_hotted_label = np.eye(num_classes)[dummy_label]
        yield one_hotted_label # shape: (batch_size, num_classes)


In [11]:
for i in data_gen():
    print(i)
    break

TypeError: 'list' object cannot be interpreted as an integer

# Dummy training data/label generator (batched)

In [12]:
epochs = 2
num_data = 50000
batch_size = 64
steps_per_epoch = num_data // batch_size

def data_gen():
    for i in range(steps_per_epoch):
        shape = (batch_size,) + input_shape
        dummy_img = np.random.rand(*shape).astype('float32')
        yield dummy_img # shape: (64,112,112,3)

def label_gen():
    for i in range(steps_per_epoch):
        dummy_label = np.random.randint(
            0, num_classes,
            (batch_size,)
        ) # shape: (64)
        one_hotted_label = np.eye(num_classes)[dummy_label]
        yield one_hotted_label # shape: (batch_size, num_classes)


In [13]:
for i in data_gen():
    print(i)
    break

[[[[9.10758898e-02 8.60775828e-01 8.61020684e-01]
   [7.76243985e-01 6.05471492e-01 2.41320193e-01]
   [3.03076684e-01 8.92413318e-01 6.39101803e-01]
   ...
   [1.62526757e-01 1.79390937e-01 5.00060916e-01]
   [7.98140764e-01 9.82698202e-02 7.88304567e-01]
   [5.84975243e-01 5.14024794e-01 4.05379027e-01]]

  [[8.72694194e-01 4.37491298e-01 5.80110908e-01]
   [2.61794686e-01 8.52178633e-01 9.92005527e-01]
   [7.25997806e-01 3.83984715e-01 2.33072955e-02]
   ...
   [2.68256664e-01 8.94986689e-01 6.01110518e-01]
   [2.73085415e-01 1.16445094e-01 4.33490425e-01]
   [3.18308443e-01 2.10291371e-01 1.46284569e-02]]

  [[5.44286072e-01 8.05564344e-01 5.33425331e-01]
   [6.89611733e-01 3.34191054e-01 8.30124378e-01]
   [1.22990720e-01 7.39497244e-01 4.02599931e-01]
   ...
   [7.18702003e-02 8.32419038e-01 5.26114941e-01]
   [7.79844642e-01 9.49193180e-01 8.70682478e-01]
   [5.33927381e-02 7.72312164e-01 7.17351735e-01]]

  ...

  [[3.19752358e-02 6.68994606e-01 5.28963149e-01]
   [4.01022971e-

In [14]:
for i in data_gen():
    print(i.shape)
    break

(64, 112, 112, 3)


In [15]:
for i in label_gen():
    print(i.shape)
    break

NameError: name 'num_classes' is not defined

In [16]:
num_classes = 500

In [17]:
for i in label_gen():
    print(i.shape)
    break

(64, 500)


In [18]:
for i in label_gen():
    print(i)
    break

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# Dummy training data/label generator (batched)

In [19]:
epochs = 2
num_data = 50000
batch_size = 64
steps_per_epoch = num_data // batch_size

def data_gen():
    for i in range(steps_per_epoch):
        shape = (batch_size,) + input_shape
        dummy_img = np.random.rand(*shape).astype('float32')
        yield dummy_img # shape: (64,112,112,3)

def label_gen():
    for i in range(steps_per_epoch):
        dummy_label = np.random.randint(
            0, num_classes,
            (batch_size,)
        ) # shape: (64)
        one_hotted_label = np.eye(num_classes)[dummy_label]
        yield one_hotted_label # shape: (64, 500)


In [20]:
dt_train = tf.data.Dataset.from_generator(data_gen)


NameError: name 'tf' is not defined

In [21]:
import tensorflow as tf

In [22]:
dt_train = tf.data.Dataset.from_generator(data_gen)


TypeError: from_generator() missing 1 required positional argument: 'output_types'

In [23]:
dt_train = tf.data.Dataset.from_generator(
    data_gen,
    output_types = [tf.float32]
)


TypeError: unhashable type: 'list'

In [24]:
dt_train = tf.data.Dataset.from_generator(
    data_gen,
    output_types = (tf.float32)
)


InvalidArgumentError: device CUDA:0 not supported by XLA service
	while setting up XLA_GPU_JIT device number 0

# Import modules

In [1]:
import tensorflow as tf
from facelib.dev.layer import DenseL2
from facelib.dev.loss import ArcFace
from facelib.dev.model import mobile_face_net
import numpy as np


ModuleNotFoundError: No module named 'facelib'

# Prepare model architecture

In [2]:
input_shape = (112,112,3)
num_classes = 500
num_features = 128
model = mobile_face_net(input_shape = input_shape)
x = model.output
x = tf.keras.layers.BatchNormalization(name='features')(x)
x = DenseL2(num_classes)(x)
model = tf.keras.models.Model(model.input, x)


NameError: name 'mobile_face_net' is not defined

# Compile model

In [3]:
af = ArcFace(m=0.4, scale=30)
losses = [af.loss]
optimizer = tf.keras.optimizers.Adam()
metrics = [
    tf.keras.metrics.CategoricalAccuracy(name='acc'),
]
model.compile(optimizer=optimizer, loss=losses, metrics=metrics)


NameError: name 'ArcFace' is not defined

# Dummy training data/label generator (batched)

In [4]:
epochs = 2
num_data = 50000
batch_size = 64
steps_per_epoch = num_data // batch_size

def data_gen():
    for i in range(steps_per_epoch):
        shape = (batch_size,) + input_shape
        dummy_img = np.random.rand(*shape).astype('float32')
        yield dummy_img # shape: (64,112,112,3)

def label_gen():
    for i in range(steps_per_epoch):
        dummy_label = np.random.randint(
            0, num_classes,
            (batch_size,)
        ) # shape: (64)
        one_hotted_label = np.eye(num_classes)[dummy_label]
        yield one_hotted_label # shape: (64, 500)


# Convert to tfdata

In [5]:
dt_train = tf.data.Dataset.from_generator(
    data_gen,
    output_types = (tf.float32)
)
dt_train = dt_train.


SyntaxError: invalid syntax (<ipython-input-5-b24e7ac99626>, line 5)

In [6]:
dt_train

NameError: name 'dt_train' is not defined

In [7]:
dt_train = tf.data.Dataset.from_generator(
    data_gen,
    output_types = (tf.float32)
)


In [8]:
for i in dt_train:
    print(i)
    break

UnknownError: NameError: name 'np' is not defined
Traceback (most recent call last):

  File "/home/kutay/envs/tensorflow-2.1/lib/python3.6/site-packages/tensorflow_core/python/ops/script_ops.py", line 236, in __call__
    ret = func(*args)

  File "/home/kutay/envs/tensorflow-2.1/lib/python3.6/site-packages/tensorflow_core/python/data/ops/dataset_ops.py", line 789, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "<ipython-input-4-0d2887c4ca26>", line 9, in data_gen
    dummy_img = np.random.rand(*shape).astype('float32')

NameError: name 'np' is not defined


	 [[{{node PyFunc}}]]

In [9]:
import numpy as np

In [10]:
## Convert to tfdata

dt_train = tf.data.Dataset.from_generator(
    data_gen,
    output_types = (tf.float32)
)


In [11]:
for i in dt_train:
    print(i)
    break

tf.Tensor(
[[[[0.07797478 0.4815548  0.8346347 ]
   [0.44651252 0.08030438 0.5622932 ]
   [0.39759567 0.73332435 0.8580381 ]
   ...
   [0.85607356 0.5524594  0.4678752 ]
   [0.11132263 0.8747459  0.16170128]
   [0.2106826  0.06529126 0.11444204]]

  [[0.4769326  0.9970873  0.68204457]
   [0.03693701 0.07345631 0.3803322 ]
   [0.9826151  0.25343087 0.5692786 ]
   ...
   [0.84625053 0.9622234  0.42778423]
   [0.82375276 0.38682967 0.6163153 ]
   [0.7546638  0.45071694 0.37169266]]

  [[0.9691563  0.86979634 0.12876378]
   [0.60532296 0.24515215 0.9427024 ]
   [0.23043317 0.4720839  0.7841033 ]
   ...
   [0.7247739  0.27756286 0.00823263]
   [0.63172704 0.41600245 0.41708943]
   [0.33803198 0.02009802 0.45197055]]

  ...

  [[0.41731685 0.3215111  0.24020654]
   [0.2502045  0.27907372 0.39006048]
   [0.00732114 0.05878767 0.8079319 ]
   ...
   [0.3359829  0.98022383 0.9544334 ]
   [0.4206245  0.21069323 0.9736141 ]
   [0.30383697 0.3148778  0.76989144]]

  [[0.08441998 0.37228712 0.475524

In [12]:
for i in dt_train:
    print(i.shape)
    break

(64, 112, 112, 3)


In [13]:
for i in label_gen():
    a = i
    break

In [14]:
a

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
a.dtype

dtype('float64')

# Dummy training data/label generator (batched)

In [16]:
num_epochs = 2
num_data = 50000
batch_size = 64
steps_per_epoch = num_data // batch_size

def data_gen():
    for i in range(steps_per_epoch):
        shape = (batch_size,) + input_shape
        dummy_img = np.random.rand(*shape).astype('float32')
        yield dummy_img # shape: (64,112,112,3)

def label_gen():
    for i in range(steps_per_epoch):
        dummy_label = np.random.randint(
            0, num_classes,
            (batch_size,)
        ) # shape: (64)
        one_hotted_label = np.eye(num_classes, dtype='int32')[dummy_label]
        yield one_hotted_label # shape: (64, 500)


In [17]:
for i in label_gen():
    a = i
    break

In [18]:
a.dtype

dtype('int32')

In [19]:
a

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [20]:
pow(2, 32)

4294967296

# Convert to tfdata

In [21]:
dt_train = tf.data.Dataset.from_generator(
    data_gen,
    output_types = (tf.float32)
)
dt_train = dt_train.repeat(num_epochs)
dt_train = dt_train.prefetch(2)

dt_label = tf.data.Dataset.from_generator(
    label_gen,
    output_types = (tf.int32)
)
dt_label = dt_label.repeat(num_epochs)
dt_label = dt_label.prefetch(2)


In [22]:
for i in dt_label:
    a = i
    break

In [23]:
a

<tf.Tensor: shape=(64, 500), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>

In [24]:
a.dtype

tf.int32

In [25]:
dt_train = tf.data.Dataset.zip((dt_data, dt_label))


NameError: name 'dt_data' is not defined

# Convert to tfdata

In [26]:
dt_data = tf.data.Dataset.from_generator(
    data_gen,
    output_types = (tf.float32)
)
dt_data = dt_train.repeat(num_epochs)
dt_data = dt_train.prefetch(2)

dt_label = tf.data.Dataset.from_generator(
    label_gen,
    output_types = (tf.int32)
)
dt_label = dt_label.repeat(num_epochs)
dt_label = dt_label.prefetch(2)

dt_train = tf.data.Dataset.zip((dt_data, dt_label))


# Convert to tfdata

In [27]:
dt_data = tf.data.Dataset.from_generator(
    data_gen,
    output_types = (tf.float32)
)
dt_data = dt_data.repeat(num_epochs)
dt_data = dt_data.prefetch(2)

dt_label = tf.data.Dataset.from_generator(
    label_gen,
    output_types = (tf.int32)
)
dt_label = dt_label.repeat(num_epochs)
dt_label = dt_label.prefetch(2)

dt_train = tf.data.Dataset.zip((dt_data, dt_label))


In [28]:
for i in dt_train:
    a = i
    break

In [29]:
a

(<tf.Tensor: shape=(64, 112, 112, 3), dtype=float32, numpy=
 array([[[[8.33280265e-01, 2.76231061e-04, 3.38525981e-01],
          [7.46178448e-01, 5.73110998e-01, 1.51439920e-01],
          [2.92967111e-01, 5.00603676e-01, 7.71542564e-02],
          ...,
          [8.49965274e-01, 9.13599133e-01, 6.84665859e-01],
          [6.97658479e-01, 9.20013607e-01, 4.00767088e-01],
          [9.67579305e-01, 2.22800253e-03, 4.57318962e-01]],
 
         [[1.23988368e-01, 7.37500668e-01, 1.17807880e-01],
          [2.19465658e-01, 2.36634836e-01, 7.70424366e-01],
          [4.25550818e-01, 2.62953877e-01, 5.53570628e-01],
          ...,
          [4.48588341e-01, 2.64999092e-01, 1.83066711e-01],
          [3.44834000e-01, 5.58244050e-01, 9.76561368e-01],
          [6.06422067e-01, 9.76352811e-01, 8.85257781e-01]],
 
         [[3.37753028e-01, 3.25244159e-01, 3.13912183e-01],
          [7.94575214e-01, 9.63929594e-02, 1.84725866e-01],
          [8.41956794e-01, 8.34679544e-01, 6.74708545e-01],
    

In [30]:
a.shape

AttributeError: 'tuple' object has no attribute 'shape'

In [31]:
tf.shape(a)

InvalidArgumentError: cannot compute Pack as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:Pack] name: packed

In [32]:
a

(<tf.Tensor: shape=(64, 112, 112, 3), dtype=float32, numpy=
 array([[[[8.33280265e-01, 2.76231061e-04, 3.38525981e-01],
          [7.46178448e-01, 5.73110998e-01, 1.51439920e-01],
          [2.92967111e-01, 5.00603676e-01, 7.71542564e-02],
          ...,
          [8.49965274e-01, 9.13599133e-01, 6.84665859e-01],
          [6.97658479e-01, 9.20013607e-01, 4.00767088e-01],
          [9.67579305e-01, 2.22800253e-03, 4.57318962e-01]],
 
         [[1.23988368e-01, 7.37500668e-01, 1.17807880e-01],
          [2.19465658e-01, 2.36634836e-01, 7.70424366e-01],
          [4.25550818e-01, 2.62953877e-01, 5.53570628e-01],
          ...,
          [4.48588341e-01, 2.64999092e-01, 1.83066711e-01],
          [3.44834000e-01, 5.58244050e-01, 9.76561368e-01],
          [6.06422067e-01, 9.76352811e-01, 8.85257781e-01]],
 
         [[3.37753028e-01, 3.25244159e-01, 3.13912183e-01],
          [7.94575214e-01, 9.63929594e-02, 1.84725866e-01],
          [8.41956794e-01, 8.34679544e-01, 6.74708545e-01],
    

# Initiate training

In [33]:
model.fit(
    dt_train,
    steps_per_epoch=steps_per_epoch,
    epochs=num_epochs,
    verbose=1,
)


NameError: name 'model' is not defined

# Import modules

# Import modules

In [5]:
import tensorflow as tf
from facelib.dev.layer import DenseL2
from facelib.dev.loss import ArcFace
from facelib.dev.model import mobile_face_net
import numpy as np


# Prepare model architecture

In [6]:
input_shape = (112,112,3)
num_classes = 500
num_features = 128
model = mobile_face_net(input_shape = input_shape)
x = model.output
x = tf.keras.layers.BatchNormalization(name='features')(x)
x = DenseL2(num_classes)(x)
model = tf.keras.models.Model(model.input, x)


# Compile model

In [7]:
af = ArcFace(m=0.4, scale=30)
losses = [af.loss]
optimizer = tf.keras.optimizers.Adam()
metrics = [
    tf.keras.metrics.CategoricalAccuracy(name='acc'),
]
model.compile(optimizer=optimizer, loss=losses, metrics=metrics)


# Print model architecture

In [8]:
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 112, 112, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 56, 56, 64)        1792      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 56, 56, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 56, 56, 64)        256       
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 56, 56, 64)        640       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 56, 56, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 64)        256   

# Dummy training data/label generator (batched)

In [9]:
num_epochs = 2
num_data = 50000
batch_size = 64
steps_per_epoch = num_data // batch_size

def data_gen():
    for i in range(steps_per_epoch):
        shape = (batch_size,) + input_shape
        dummy_img = np.random.rand(*shape).astype('float32')
        yield dummy_img # shape: (64,112,112,3)

def label_gen():
    for i in range(steps_per_epoch):
        dummy_label = np.random.randint(
            0, num_classes,
            (batch_size,)
        ) # shape: (64)
        one_hotted_label = np.eye(num_classes, dtype='int32')[dummy_label]
        yield one_hotted_label # shape: (64,500)


# Convert to tfdata

In [10]:
dt_data = tf.data.Dataset.from_generator(
    data_gen,
    output_types = (tf.float32)
)
dt_data = dt_data.repeat(num_epochs)
dt_data = dt_data.prefetch(2)

dt_label = tf.data.Dataset.from_generator(
    label_gen,
    output_types = (tf.int32)
)
dt_label = dt_label.repeat(num_epochs)
dt_label = dt_label.prefetch(2)

# shape: ((64,112,112,3), (64,500)
dt_train = tf.data.Dataset.zip((dt_data, dt_label))


# Initiate training

In [11]:
model.fit(
    dt_train,
    steps_per_epoch=steps_per_epoch,
    epochs=num_epochs,
    verbose=1,
)


ValueError: in converted code:

    /home/kutay/envs/tensorflow-2.1/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training_v2.py:677 map_fn
        batch_size=None)
    /home/kutay/envs/tensorflow-2.1/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training.py:2410 _standardize_tensors
        exception_prefix='input')
    /home/kutay/envs/tensorflow-2.1/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training_utils.py:526 standardize_input_data
        standardize_single_array(x, shape) for (x, shape) in zip(data, shapes)
    /home/kutay/envs/tensorflow-2.1/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training_utils.py:526 <listcomp>
        standardize_single_array(x, shape) for (x, shape) in zip(data, shapes)
    /home/kutay/envs/tensorflow-2.1/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training_utils.py:451 standardize_single_array
        if (x.shape is not None and len(x.shape) == 1 and
    

In [12]:
dt_data = tf.data.Dataset.from_generator(
    data_gen,
    output_types = (tf.float32),
    output_shapes = (64, 500)
)


# Convert to tfdata

In [13]:
dt_data = tf.data.Dataset.from_generator(
    data_gen,
    output_types = (tf.float32),
    output_shapes = (64, 112, 112, 3)
)
dt_data = dt_data.repeat(num_epochs)
dt_data = dt_data.prefetch(2)

dt_label = tf.data.Dataset.from_generator(
    label_gen,
    output_types = (tf.int32),
    output_shapes = (64, 500)
)
dt_label = dt_label.repeat(num_epochs)
dt_label = dt_label.prefetch(2)

# shape: ((64,112,112,3), (64,500)
dt_train = tf.data.Dataset.zip((dt_data, dt_label))


# Initiate training

In [14]:
model.fit(
    dt_train,
    steps_per_epoch=steps_per_epoch,
    epochs=num_epochs,
    verbose=1,
)


Train for 781 steps
Epoch 1/2
  1/781 [..............................] - ETA: 5:25

TypeError: Input 'y' of 'Mul' Op has type float32 that does not match type int32 of argument 'x'.

# Dummy training data/label generator (batched)

In [15]:
num_epochs = 2
num_data = 50000
batch_size = 64
steps_per_epoch = num_data // batch_size

def data_gen():
    for i in range(steps_per_epoch):
        shape = (batch_size,) + input_shape
        dummy_img = np.random.rand(*shape).astype('float32')
        yield dummy_img # shape: (64,112,112,3)

def label_gen():
    for i in range(steps_per_epoch):
        dummy_label = np.random.randint(
            0, num_classes,
            (batch_size,)
        ) # shape: (64)
        one_hotted_label = np.eye(num_classes, dtype='float32')[dummy_label]
        yield one_hotted_label # shape: (64,500)


# Convert to tfdata

In [16]:
dt_data = tf.data.Dataset.from_generator(
    data_gen,
    output_types = (tf.float32),
    output_shapes = (64, 112, 112, 3)
)
dt_data = dt_data.repeat(num_epochs)
dt_data = dt_data.prefetch(2)

dt_label = tf.data.Dataset.from_generator(
    label_gen,
    output_types = (tf.int32),
    output_shapes = (64, 500)
)
dt_label = dt_label.repeat(num_epochs)
dt_label = dt_label.prefetch(2)

# shape: ((64,112,112,3), (64,500)
dt_train = tf.data.Dataset.zip((dt_data, dt_label))


# Convert to tfdata

In [17]:
dt_data = tf.data.Dataset.from_generator(
    data_gen,
    output_types = (tf.float32),
    output_shapes = (64, 112, 112, 3)
)
dt_data = dt_data.repeat(num_epochs)
dt_data = dt_data.prefetch(2)

dt_label = tf.data.Dataset.from_generator(
    label_gen,
    output_types = (tf.float32),
    output_shapes = (64, 500)
)
dt_label = dt_label.repeat(num_epochs)
dt_label = dt_label.prefetch(2)

# shape: ((64,112,112,3), (64,500)
dt_train = tf.data.Dataset.zip((dt_data, dt_label))


# Initiate training

In [18]:
model.fit(
    dt_train,
    steps_per_epoch=steps_per_epoch,
    epochs=num_epochs,
    verbose=1,
)


Train for 781 steps
Epoch 1/2
  1/781 [..............................] - ETA: 43:54

ValueError: Creating variables on a non-first call to a function decorated with tf.function.

# Import modules

# Import modules

In [3]:
import tensorflow as tf
from facelib.dev.layer import DenseL2
from facelib.dev.loss import ArcFace
from facelib.dev.model import mobile_face_net
import numpy as np


# Prepare model architecture

In [4]:
input_shape = (112,112,3)
num_classes = 500
num_features = 128
model = mobile_face_net(input_shape = input_shape)
x = model.output
x = tf.keras.layers.BatchNormalization(name='features')(x)
x = DenseL2(num_classes)(x)
model = tf.keras.models.Model(model.input, x)


# Compile model

In [5]:
af = ArcFace(m=0.4, scale=30)
losses = [af.loss]
optimizer = tf.keras.optimizers.Adam()
metrics = [
    tf.keras.metrics.CategoricalAccuracy(name='acc'),
]
model.compile(optimizer=optimizer, loss=losses, metrics=metrics)


# Print model architecture

In [6]:
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 112, 112, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 56, 56, 64)        1792      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 56, 56, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 56, 56, 64)        256       
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 56, 56, 64)        640       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 56, 56, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 64)        256   

# Dummy training data/label generator (batched)

In [7]:
num_epochs = 2
num_data = 50000
batch_size = 64
steps_per_epoch = num_data // batch_size

def data_gen():
    for i in range(steps_per_epoch):
        shape = (batch_size,) + input_shape
        dummy_img = np.random.rand(*shape).astype('float32')
        yield dummy_img # shape: (64,112,112,3)

def label_gen():
    for i in range(steps_per_epoch):
        dummy_label = np.random.randint(
            0, num_classes,
            (batch_size,)
        ) # shape: (64)
        one_hotted_label = np.eye(num_classes, dtype='float32')[dummy_label]
        yield one_hotted_label # shape: (64,500)


# Convert to tfdata

In [8]:
dt_data = tf.data.Dataset.from_generator(
    data_gen,
    output_types = (tf.float32),
    output_shapes = (64, 112, 112, 3)
)
dt_data = dt_data.repeat(num_epochs)
dt_data = dt_data.prefetch(2)

dt_label = tf.data.Dataset.from_generator(
    label_gen,
    output_types = (tf.float32),
    output_shapes = (64, 500)
)
dt_label = dt_label.repeat(num_epochs)
dt_label = dt_label.prefetch(2)

# shape: ((64,112,112,3), (64,500)
dt_train = tf.data.Dataset.zip((dt_data, dt_label))


# Initiate training

In [9]:
model.fit(
    dt_train,
    steps_per_epoch=steps_per_epoch,
    epochs=num_epochs,
    verbose=1,
)


Train for 781 steps
Epoch 1/2
 28/781 [>.............................] - ETA: 21:44 - loss: 21.1033 - acc: 0.0029

KeyboardInterrupt: 

# Import modules

In [3]:
"""Prepare mobilefacenet model with arcface loss."""
#%% [markdown]
## Import modules
import tensorflow as tf
from facelib.dev.layer import DenseL2
from facelib.dev.loss import ArcFace
from facelib.dev.model import mobile_face_net
import numpy as np


# Prepare model architecture

In [4]:
input_shape = (112,112,3)
num_classes = 500
num_features = 128
model = mobile_face_net(input_shape = input_shape)
x = model.output
x = tf.keras.layers.BatchNormalization(name='features')(x)
x = DenseL2(num_classes)(x)
model = tf.keras.models.Model(model.input, x)


# Compile model

In [5]:
af = ArcFace(m=0.4, scale=30)
losses = [af.loss]
optimizer = tf.keras.optimizers.Adam()
metrics = [
    tf.keras.metrics.CategoricalAccuracy(name='acc'),
]
model.compile(optimizer=optimizer, loss=losses, metrics=metrics)


# Print model architecture

In [6]:
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 112, 112, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 56, 56, 64)        1792      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 56, 56, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 56, 56, 64)        256       
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 56, 56, 64)        640       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 56, 56, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 64)        256   

# Dummy training data/label generator (batched)

In [7]:
num_epochs = 2
num_data = 5000
batch_size = 64
steps_per_epoch = num_data // batch_size

def data_gen():
    for i in range(steps_per_epoch):
        shape = (batch_size,) + input_shape
        dummy_img = np.random.rand(*shape).astype('float32')
        yield dummy_img # shape: (64,112,112,3)

def label_gen():
    for i in range(steps_per_epoch):
        dummy_label = np.random.randint(
            0, num_classes,
            (batch_size,)
        ) # shape: (64)
        one_hotted_label = np.eye(num_classes, dtype='float32')[dummy_label]
        yield one_hotted_label # shape: (64,500)


# Convert to tfdata

In [8]:
dt_data = tf.data.Dataset.from_generator(
    data_gen,
    output_types = (tf.float32),
    output_shapes = (64, 112, 112, 3)
)
dt_data = dt_data.repeat(num_epochs)
dt_data = dt_data.prefetch(2)

dt_label = tf.data.Dataset.from_generator(
    label_gen,
    output_types = (tf.float32),
    output_shapes = (64, 500)
)
dt_label = dt_label.repeat(num_epochs)
dt_label = dt_label.prefetch(2)

# shape: ((64,112,112,3), (64,500)
dt_train = tf.data.Dataset.zip((dt_data, dt_label))


# Initiate training

In [9]:
model.fit(
    dt_train,
    steps_per_epoch=steps_per_epoch,
    epochs=num_epochs,
    verbose=1,
)


Train for 78 steps
Epoch 1/2
78/78 [==============================] - 130s 2s/step - loss: 20.9777 - acc: 0.0014
Epoch 2/2
78/78 [==============================] - 128s 2s/step - loss: 20.4622 - acc: 0.0020
